In [ ]:
import os
# The latest version of spark 3.2  from http://www.apache.org/dist/spark/ 
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
--2022-11-26 00:31:21--  https://jdbc.postgresql.or

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("WatchReviewsETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watch_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
watch_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
# Drop null values, duplicates, and count number of rows.
watch_df = watch_df.dropna()
watch_df = watch_df.dropDuplicates()
watch_df.count()

960679

In [ ]:
# Show schema to see if it matches the sql schema tables.
watch_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create dataframe for review_id_table schema
review_id_df = watch_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Convert review_date from string to date, check schema.
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType
review_id_df = review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))
review_id_df.printSchema()
  

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Check df to ensure it is in yyyy-mm-dd
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create products_df
products_df = watch_df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Create customers_df, need to create customer count column
customers_df = watch_df.select(["customer_id"])
customers_df = customers_df.groupBy("customer_id").count()
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
# Cast count as integer and rename to customer_count
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Preview customers_df
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Create vine_df
vine_df = watch_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Check vine_df schema
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://bigdataassignmentdb.ccobpfsx4sbf.us-east-1.rds.amazonaws.com:5432/bigdataassignmentdb"
config = {"user":"",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)